In [53]:
import pandas as pd
from tqdm import tqdm
lyrics = pd.read_csv('/Users/zhenyabudnyk/PycharmProjects/LyrAIX/Thesis/Research Questions/RQ1/prompts.csv')

In [54]:
lyrics

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Valence,Arousal,Dominance,Part,Artist,Topic,Explicitness,Scheme,Instruction,results,predicted explicitness,predicted scheme,lemmatized,"tokenized, lemmatized, no bad words",topic,topic vectors
0,0,0,0,High,Low,Medium,Chorus,Sofya Wang,life and relationships,Normal,0-0-0-1-0-0-1-1,Generate a Chorus of song lyrics in the style ...,"We're lying, we're lying in the shadows\nI'm f...",Normal,0-1-2-3-4-5-6-7-8-9,"['lying', 'lying', 'the', 'shadow', 'flying', ...","['lying', 'lying', 'shadow', 'flying', 'flying...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
1,1,1,1,High,Medium,Low,Bridge,Chiddy Bang,life and relationships,Normal,0-0-1-2-3-4-5-6,Generate a Bridge of song lyrics in the style ...,You don't wanna party\nWhy you tryna party\nI ...,Normal,0-1-2-3-4-2-4-5,"['you', 'don', 'wanna', 'party', 'why', 'you',...","['wanna', 'party', 'tryna', 'party', 'even', '...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
2,2,2,2,High,Medium,High,Chorus,Shwayze,life and relationships,Normal,0-1-2-0-3-3-4,Generate a Chorus of song lyrics in the style ...,"And if you run from me, baby, run from me\nYou...",Normal,0-1-2-3-4-5-4,"['and', 'you', 'run', 'from', 'baby', 'run', '...","['run', 'baby', 'run', 'could', 'live', 'good'...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
3,3,3,3,Medium,Low,High,Bridge,Dove Cameron,general,Normal,0-1-2-1-1-1,Generate a Bridge of song lyrics in the style ...,"Swing on by, all you can hear is me\nLet's get...",Normal,0-1-2-3-4-5,"['swing', 'all', 'you', 'can', 'hear', 'let', ...","['swing', 'hear', 'let', 'get', 'party', 'star...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
4,4,4,4,Medium,High,Medium,Chorus,Weezer,general,Normal,0,Generate a Chorus of song lyrics in the style ...,Singing like a symphony in a restaurant\n,Normal,0,"['singing', 'like', 'symphony', 'restaurant']","['singing', 'like', 'symphony', 'restaurant']",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,295,295,High,High,Low,Verse,The Irish Rovers,general,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18...,Generate a Verse of song lyrics in the style o...,Down to the wire\nI'm down to the wire\nI'm do...,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18,"['down', 'the', 'wire', 'down', 'the', 'wire',...","['wire', 'wire', 'wire', 'wire', 'wire', 'wire...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
296,296,296,296,High,Low,High,Verse,Genius English Translations,life and relationships,Explicit content,0-0-0-0,Generate a Verse of song lyrics in the style o...,"Well, I want to taste my eyes\nI wanna see the...",Normal,0-0-1-2,"['well', 'want', 'taste', 'eye', 'wanna', 'see...","['well', 'want', 'taste', 'eye', 'wanna', 'see...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
297,297,297,297,Low,High,Medium,Verse,Tiana Major9 & SiR,general,Normal,0-1-2-1,Generate a Verse of song lyrics in the style o...,"It's a big world, but I gotta do my thing\nThe...",Normal,0-1-2-3,"['big', 'world', 'but', 'gotta', 'thing', 'the...","['big', 'world', 'gotta', 'thing', 'see', 'lot...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
298,298,298,298,Medium,Medium,Low,Verse,Spacey Jane,religion and society,Explicit content,0-1-2-3,Generate a Verse of song lyrics in the style o...,Him to me a mwah a cryd for the people that kn...,Normal,0-0-0-0,"['him', 'mwah', 'cryd', 'for', 'the', 'people'...","['mwah', 'cryd', 'people', 'know', 'world', 'w...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."


In [55]:
lyrics_corpus_tokenized = list()
for song in tqdm(lyrics['tokenized, lemmatized, no bad words'], desc="writing to the list"):
    lyrics_corpus_tokenized.append(song.strip("[]").replace("'", "").split(", "))
print(type(lyrics_corpus_tokenized[0]))
print(lyrics_corpus_tokenized[0])

writing to the list: 100%|██████████| 300/300 [00:00<00:00, 27012.39it/s]

<class 'list'>
['lying', 'lying', 'shadow', 'flying', 'flying', 'sky', 'stop', 'wondering', 'got', 'beautiful', 'beautiful', 'stop', 'chasing', 'dream', 'lying', 'lying', 'shadow', 'flying', 'flying', 'sky', 'stop', 'wondering', 'got', 'beautiful', 'beautiful', 'stop', 'chasing', 'dream']


In [56]:
lyrics

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Valence,Arousal,Dominance,Part,Artist,Topic,Explicitness,Scheme,Instruction,results,predicted explicitness,predicted scheme,lemmatized,"tokenized, lemmatized, no bad words",topic,topic vectors
0,0,0,0,High,Low,Medium,Chorus,Sofya Wang,life and relationships,Normal,0-0-0-1-0-0-1-1,Generate a Chorus of song lyrics in the style ...,"We're lying, we're lying in the shadows\nI'm f...",Normal,0-1-2-3-4-5-6-7-8-9,"['lying', 'lying', 'the', 'shadow', 'flying', ...","['lying', 'lying', 'shadow', 'flying', 'flying...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
1,1,1,1,High,Medium,Low,Bridge,Chiddy Bang,life and relationships,Normal,0-0-1-2-3-4-5-6,Generate a Bridge of song lyrics in the style ...,You don't wanna party\nWhy you tryna party\nI ...,Normal,0-1-2-3-4-2-4-5,"['you', 'don', 'wanna', 'party', 'why', 'you',...","['wanna', 'party', 'tryna', 'party', 'even', '...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
2,2,2,2,High,Medium,High,Chorus,Shwayze,life and relationships,Normal,0-1-2-0-3-3-4,Generate a Chorus of song lyrics in the style ...,"And if you run from me, baby, run from me\nYou...",Normal,0-1-2-3-4-5-4,"['and', 'you', 'run', 'from', 'baby', 'run', '...","['run', 'baby', 'run', 'could', 'live', 'good'...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
3,3,3,3,Medium,Low,High,Bridge,Dove Cameron,general,Normal,0-1-2-1-1-1,Generate a Bridge of song lyrics in the style ...,"Swing on by, all you can hear is me\nLet's get...",Normal,0-1-2-3-4-5,"['swing', 'all', 'you', 'can', 'hear', 'let', ...","['swing', 'hear', 'let', 'get', 'party', 'star...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
4,4,4,4,Medium,High,Medium,Chorus,Weezer,general,Normal,0,Generate a Chorus of song lyrics in the style ...,Singing like a symphony in a restaurant\n,Normal,0,"['singing', 'like', 'symphony', 'restaurant']","['singing', 'like', 'symphony', 'restaurant']",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,295,295,High,High,Low,Verse,The Irish Rovers,general,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18...,Generate a Verse of song lyrics in the style o...,Down to the wire\nI'm down to the wire\nI'm do...,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18,"['down', 'the', 'wire', 'down', 'the', 'wire',...","['wire', 'wire', 'wire', 'wire', 'wire', 'wire...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
296,296,296,296,High,Low,High,Verse,Genius English Translations,life and relationships,Explicit content,0-0-0-0,Generate a Verse of song lyrics in the style o...,"Well, I want to taste my eyes\nI wanna see the...",Normal,0-0-1-2,"['well', 'want', 'taste', 'eye', 'wanna', 'see...","['well', 'want', 'taste', 'eye', 'wanna', 'see...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
297,297,297,297,Low,High,Medium,Verse,Tiana Major9 & SiR,general,Normal,0-1-2-1,Generate a Verse of song lyrics in the style o...,"It's a big world, but I gotta do my thing\nThe...",Normal,0-1-2-3,"['big', 'world', 'but', 'gotta', 'thing', 'the...","['big', 'world', 'gotta', 'thing', 'see', 'lot...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."
298,298,298,298,Medium,Medium,Low,Verse,Spacey Jane,religion and society,Explicit content,0-1-2-3,Generate a Verse of song lyrics in the style o...,Him to me a mwah a cryd for the people that kn...,Normal,0-0-0-0,"['him', 'mwah', 'cryd', 'for', 'the', 'people'...","['mwah', 'cryd', 'people', 'know', 'world', 'w...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915..."


In [16]:
# words_to_remove = {'verse', 'chorus', 'hook', 'bridge', 'intro', 'outro', 'pre-chorus', 'pre-hook', 'get', 'got'}


In [17]:
# lyrics_corpus_tokenized = [
#     [token for token in song if token not in words_to_remove]
#     for song in tqdm(lyrics_corpus_tokenized, desc="removing")
# ]

removing: 100%|██████████| 300/300 [00:00<00:00, 307951.84it/s]


In [57]:
from gensim.corpora import Dictionary
from tqdm import tqdm
import random

# Assuming you have a big list called 'big_list'
  # Number of items to choose

# Randomly select 20,000 items from the big list
# lyric_corpus_tokenized = lyric_corpus_tokenized.tolist()

tqdm_corpus = tqdm(lyrics_corpus_tokenized, desc="Creating dictionary")
dictionary = Dictionary(tqdm_corpus)

Creating dictionary: 100%|██████████| 300/300 [00:00<00:00, 37684.67it/s]


In [58]:
# dictionary.filter_extremes(no_below=100, no_above=0.8)

In [59]:
from gensim.corpora import MmCorpus
gensim_corpus = []
with tqdm(total=len(lyrics_corpus_tokenized), desc="Creating Gensim Corpus") as pbar:
    for song in lyrics_corpus_tokenized:
        doc = dictionary.doc2bow(song)
        gensim_corpus.append(doc)
        pbar.update(1)
temp = dictionary[0]
id2word = dictionary.id2token

Creating Gensim Corpus: 100%|██████████| 300/300 [00:00<00:00, 65216.71it/s]


In [60]:
dictionary[1]

'chasing'

In [29]:
import time
from gensim.models import LdaModel
import random

start_time = time.time()
chunksize = 2000
passes = 20
iterations = 400
num_topics = 4
sample_size = 500000


gensim2 = random.sample(gensim_corpus, sample_size)

lda_model = LdaModel(
    corpus=gensim2,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes
)

end_time = time.time()
training_time = end_time - start_time
print("Training time: {:.2f} seconds".format(training_time))

Training time: 1124.63 seconds


In [61]:
import time
from gensim.models import LdaModel
import random
lda_model = LdaModel.load("/Users/zhenyabudnyk/PycharmProjects/LyrAIX/Thesis/Data Labeling/Topic Labeling/LDA 4 topics/lda_model_4")

In [34]:
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
# vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
# pyLDAvis.display(vis_data)

In [32]:

#pyLDAvis.save_html(vis_data, './Lyrics_LDA_'+ '4' +'.html')

In [33]:
# lda_model.save("lda_model_4")


In [47]:
topics = ['life and relationships', 'money and authority', 'general', 'religion and society']

In [48]:
predicted_topics = list()
with tqdm(total=len(gensim_corpus), desc="Predicting topics") as pbar:
    for song in gensim_corpus:
        topic_dist = lda_model.get_document_topics(song)
        predicted_topics.append(topic_dist)
        pbar.update(1)

Predicting topics: 100%|██████████| 300/300 [00:00<00:00, 4133.83it/s]


In [49]:
max_indices = []
for vector in predicted_topics:
    max_tuple = max(vector, key=lambda t: t[1])
    max_indices.append(topics[max_tuple[0]])


In [63]:
print(max_indices[:10])

['general', 'general', 'general', 'general', 'general', 'money and authority', 'general', 'general', 'general', 'general']


In [64]:
lyrics['predicted topic'] = max_indices

In [65]:
# count = lyrics['predicted topics'].value_counts()['family and religion']
# print(count, " family and religion")

count = lyrics['predicted topic'].value_counts()['life and relationships']
print(count, " life and relationships")
count = lyrics['predicted topic'].value_counts()['money and authority']
print(count, " money and authority")
count = lyrics['predicted topic'].value_counts()['general']
print(count, " general")
count = lyrics['predicted topic'].value_counts()['religion and society']
print(count, " religion and society")


62  life and relationships
21  money and authority
193  general
24  religion and society


In [37]:
lyrics['topic vectors'] = predicted_topics

In [66]:
lyrics

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Valence,Arousal,Dominance,Part,Artist,Topic,Explicitness,Scheme,Instruction,results,predicted explicitness,predicted scheme,lemmatized,"tokenized, lemmatized, no bad words",topic,topic vectors,predicted topic
0,0,0,0,High,Low,Medium,Chorus,Sofya Wang,life and relationships,Normal,0-0-0-1-0-0-1-1,Generate a Chorus of song lyrics in the style ...,"We're lying, we're lying in the shadows\nI'm f...",Normal,0-1-2-3-4-5-6-7-8-9,"['lying', 'lying', 'the', 'shadow', 'flying', ...","['lying', 'lying', 'shadow', 'flying', 'flying...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
1,1,1,1,High,Medium,Low,Bridge,Chiddy Bang,life and relationships,Normal,0-0-1-2-3-4-5-6,Generate a Bridge of song lyrics in the style ...,You don't wanna party\nWhy you tryna party\nI ...,Normal,0-1-2-3-4-2-4-5,"['you', 'don', 'wanna', 'party', 'why', 'you',...","['wanna', 'party', 'tryna', 'party', 'even', '...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
2,2,2,2,High,Medium,High,Chorus,Shwayze,life and relationships,Normal,0-1-2-0-3-3-4,Generate a Chorus of song lyrics in the style ...,"And if you run from me, baby, run from me\nYou...",Normal,0-1-2-3-4-5-4,"['and', 'you', 'run', 'from', 'baby', 'run', '...","['run', 'baby', 'run', 'could', 'live', 'good'...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
3,3,3,3,Medium,Low,High,Bridge,Dove Cameron,general,Normal,0-1-2-1-1-1,Generate a Bridge of song lyrics in the style ...,"Swing on by, all you can hear is me\nLet's get...",Normal,0-1-2-3-4-5,"['swing', 'all', 'you', 'can', 'hear', 'let', ...","['swing', 'hear', 'let', 'get', 'party', 'star...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
4,4,4,4,Medium,High,Medium,Chorus,Weezer,general,Normal,0,Generate a Chorus of song lyrics in the style ...,Singing like a symphony in a restaurant\n,Normal,0,"['singing', 'like', 'symphony', 'restaurant']","['singing', 'like', 'symphony', 'restaurant']",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,295,295,High,High,Low,Verse,The Irish Rovers,general,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18...,Generate a Verse of song lyrics in the style o...,Down to the wire\nI'm down to the wire\nI'm do...,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18,"['down', 'the', 'wire', 'down', 'the', 'wire',...","['wire', 'wire', 'wire', 'wire', 'wire', 'wire...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
296,296,296,296,High,Low,High,Verse,Genius English Translations,life and relationships,Explicit content,0-0-0-0,Generate a Verse of song lyrics in the style o...,"Well, I want to taste my eyes\nI wanna see the...",Normal,0-0-1-2,"['well', 'want', 'taste', 'eye', 'wanna', 'see...","['well', 'want', 'taste', 'eye', 'wanna', 'see...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
297,297,297,297,Low,High,Medium,Verse,Tiana Major9 & SiR,general,Normal,0-1-2-1,Generate a Verse of song lyrics in the style o...,"It's a big world, but I gotta do my thing\nThe...",Normal,0-1-2-3,"['big', 'world', 'but', 'gotta', 'thing', 'the...","['big', 'world', 'gotta', 'thing', 'see', 'lot...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
298,298,298,298,Medium,Medium,Low,Verse,Spacey Jane,religion and society,Explicit content,0-1-2-3,Generate a Verse of song lyrics in the style o...,Him to me a mwah a cryd for the people that kn...,Normal,0-0-0-0,"['him', 'mwah', 'cryd', 'for', 'the', 'people'...","['mwah', 'cryd', 'people', 'know', 'world', 'w...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",life and relationships


In [38]:
lyrics.to_csv('/Users/zhenyabudnyk/PycharmProjects/LyrAIX/Thesis/Research Questions/RQ1/prompts.csv', index=False)

In [72]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming your DataFrame is called 'df' with columns 'ground_truth' and 'predicted'
ground_truth = lyrics['Topic']
predicted = lyrics['predicted topic']

class_labels = ['life and relationships', 'money and authority', 'general', 'religion and society']  # Replace with your actual class labels

# Create the confusion matrix
cm = confusion_matrix(ground_truth, predicted, labels=class_labels)
# Create the confusion matrix
accuracy = accuracy_score(ground_truth, predicted)

print("Confusion Matrix:")
print(cm)
print("Accuracy:", accuracy)


print(cm)

Confusion Matrix:
[[15 11 62 11]
 [30  6 69  7]
 [10  4 53  4]
 [ 7  0  9  2]]
Accuracy: 0.25333333333333335
[[15 11 62 11]
 [30  6 69  7]
 [10  4 53  4]
 [ 7  0  9  2]]


In [70]:
lyrics.drop('topic')

KeyError: "['topic'] not found in axis"

In [69]:
lyrics

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Valence,Arousal,Dominance,Part,Artist,Topic,Explicitness,Scheme,Instruction,results,predicted explicitness,predicted scheme,lemmatized,"tokenized, lemmatized, no bad words",topic,topic vectors,predicted topic
0,0,0,0,High,Low,Medium,Chorus,Sofya Wang,life and relationships,Normal,0-0-0-1-0-0-1-1,Generate a Chorus of song lyrics in the style ...,"We're lying, we're lying in the shadows\nI'm f...",Normal,0-1-2-3-4-5-6-7-8-9,"['lying', 'lying', 'the', 'shadow', 'flying', ...","['lying', 'lying', 'shadow', 'flying', 'flying...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
1,1,1,1,High,Medium,Low,Bridge,Chiddy Bang,life and relationships,Normal,0-0-1-2-3-4-5-6,Generate a Bridge of song lyrics in the style ...,You don't wanna party\nWhy you tryna party\nI ...,Normal,0-1-2-3-4-2-4-5,"['you', 'don', 'wanna', 'party', 'why', 'you',...","['wanna', 'party', 'tryna', 'party', 'even', '...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
2,2,2,2,High,Medium,High,Chorus,Shwayze,life and relationships,Normal,0-1-2-0-3-3-4,Generate a Chorus of song lyrics in the style ...,"And if you run from me, baby, run from me\nYou...",Normal,0-1-2-3-4-5-4,"['and', 'you', 'run', 'from', 'baby', 'run', '...","['run', 'baby', 'run', 'could', 'live', 'good'...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
3,3,3,3,Medium,Low,High,Bridge,Dove Cameron,general,Normal,0-1-2-1-1-1,Generate a Bridge of song lyrics in the style ...,"Swing on by, all you can hear is me\nLet's get...",Normal,0-1-2-3-4-5,"['swing', 'all', 'you', 'can', 'hear', 'let', ...","['swing', 'hear', 'let', 'get', 'party', 'star...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
4,4,4,4,Medium,High,Medium,Chorus,Weezer,general,Normal,0,Generate a Chorus of song lyrics in the style ...,Singing like a symphony in a restaurant\n,Normal,0,"['singing', 'like', 'symphony', 'restaurant']","['singing', 'like', 'symphony', 'restaurant']",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,295,295,High,High,Low,Verse,The Irish Rovers,general,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18...,Generate a Verse of song lyrics in the style o...,Down to the wire\nI'm down to the wire\nI'm do...,Normal,0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18,"['down', 'the', 'wire', 'down', 'the', 'wire',...","['wire', 'wire', 'wire', 'wire', 'wire', 'wire...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
296,296,296,296,High,Low,High,Verse,Genius English Translations,life and relationships,Explicit content,0-0-0-0,Generate a Verse of song lyrics in the style o...,"Well, I want to taste my eyes\nI wanna see the...",Normal,0-0-1-2,"['well', 'want', 'taste', 'eye', 'wanna', 'see...","['well', 'want', 'taste', 'eye', 'wanna', 'see...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
297,297,297,297,Low,High,Medium,Verse,Tiana Major9 & SiR,general,Normal,0-1-2-1,Generate a Verse of song lyrics in the style o...,"It's a big world, but I gotta do my thing\nThe...",Normal,0-1-2-3,"['big', 'world', 'but', 'gotta', 'thing', 'the...","['big', 'world', 'gotta', 'thing', 'see', 'lot...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",general
298,298,298,298,Medium,Medium,Low,Verse,Spacey Jane,religion and society,Explicit content,0-1-2-3,Generate a Verse of song lyrics in the style o...,Him to me a mwah a cryd for the people that kn...,Normal,0-0-0-0,"['him', 'mwah', 'cryd', 'for', 'the', 'people'...","['mwah', 'cryd', 'people', 'know', 'world', 'w...",money and authority,"[(0, 0.30893075), (1, 0.3345277), (2, 0.205915...",life and relationships


In [76]:
for i, row in lyrics.iterrows():
    print(row['results'])
    print(row['Topic'])
    print(row['predicted topic'])
    print(row['Explicitness'])
    print(row['predicted explicitness'])


We're lying, we're lying in the shadows
I'm flying, I'm flying in the sky
Can't stop wondering what we got
It's so beautiful, it's so beautiful
I can't stop chasing dreams
We're lying, we're lying in the shadows
I'm flying, I'm flying in the sky
Can't stop wondering what we got
It's so beautiful, it's so beautiful
I can't stop chasing dreams

life and relationships
general
Normal
Normal
You don't wanna party
Why you tryna party
I ain't even gotta say no
'Cause you're probably
Not what I like
So I won't
Might just pass by
And I don't care

life and relationships
general
Normal
Normal
And if you run from me, baby, run from me
You could live a good life for me
But you can't run from me, baby, run from me
They can't keep me locked in a cage
But you can't run from me, baby, run from me
You'll be running very far from home
But you won't be gonna run far from me anyway

life and relationships
general
Normal
Normal
Swing on by, all you can hear is me
Let's get the party started and go back to 